In [1]:
# Cipher KATAN
import numpy as np
from os import urandom

plain_bits = 32
key_bits = 80
word_size = 16

if plain_bits == 32:
    LEN_L1 = 13
    LEN_L2 = 19
    X = (None, 12, 7, 8, 5, 3)
    Y = (None, 18, 7, 12, 10, 8, 3)
elif plain_bits == 48:
    LEN_L1 = 19
    LEN_L2 = 29
    X = (None, 18, 12, 15, 7, 6)
    Y = (None, 28, 19, 21, 13, 15, 6)
else:
    LEN_L1 = 25
    LEN_L2 = 39
    X = (None, 24, 15, 20, 11, 9)
    Y = (None, 38, 25, 33, 21, 14, 9)

IR = (
    1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
    0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
    1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
    0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
    0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
    1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
    0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
    1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
    0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
    1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
    1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
    0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1,
    1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
    0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
    0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
    1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0
    )

MASK_VAL = 2 ** plain_bits - 1;





def encrypt(plaintext = [], K = [], nr = 0):
  P = np.flip(plaintext, axis = 1)
  ks = np.zeros(shape = (len(P), np.max([2*nr, 80])), dtype = np.uint8)
  ks[:, :80]  = np.flip(K, axis=1)
  for i in range(80, nr*2):
      ks[:, i] = ks[:, i-80] ^ ks[:, i-61] ^ks[:, i-50] ^ks[:, i-13]  
  for i in range(nr):
      fa = P[:, LEN_L2 + X[1]]^P[:, LEN_L2+ X[2]] ^ ks[:, 2*i] ^ (P[:,LEN_L2+ X[3]] & P[:, LEN_L2+X[4]]) ^ (P[:,LEN_L2+ X[5]] & IR[i])
      fb = P[:, Y[1]]^P[:, Y[2]] ^ (P[:, Y[3]] & P[:, Y[4]]) ^ (P[:, Y[5]] & P[:, Y[6]]) ^ ks[:, 2*i+1]
      P = np.roll(P, 1, axis=1)
      P[:, 0] = fa
      P[:, LEN_L2] = fb

  return np.flip(P, axis=1)

def convert_to_binary(arr):
  X = np.zeros((len(arr) * plain_bits,len(arr[0])),dtype=np.uint8);
  for i in range(len(arr) * plain_bits):
    index = i // plain_bits;
    offset = plain_bits - (i % plain_bits) - 1;
    X[i] = (arr[index] >> offset) & 1;
  X = X.transpose();
  return(X);

def convert_from_binary(arr, _dtype=np.uint64):
  num_words = arr.shape[1]//plain_bits
  X = np.zeros((len(arr), num_words),dtype=_dtype);
  for i in range(num_words):
    for j in range(plain_bits):
        pos = plain_bits*i+j
        X[:, i] += 2**(plain_bits-1-j)*arr[:, pos]
  return(X);



def check_testvectors():
    # From https://github.com/differential-neural/An-Assessment-of-Differential-Neural-Distinguishers/blob/main/cipher/katan.py
    p = np.zeros((2, plain_bits), dtype = np.uint8)
    p[1]^=1
    k = np.zeros((2, key_bits), dtype = np.uint8)
    k[0]^=1
    C = convert_from_binary(encrypt(p,k,254)).flatten()
    assert hex(C[0]) == '0x7e1ff945'  
    assert hex(C[1]) == '0x432e61da'  


check_testvectors()





In [3]:
import time
import numpy as np

def benchmark_present80(num_tests, rounds):
    """
    Benchmark the PRESENT-80 cipher.
    :param num_tests: Number of plaintext-ciphertext pairs to test.
    :param rounds: Number of encryption rounds.
    """
    # Generate random plaintexts and keys
    plaintexts = np.random.randint(0, 2, size=(num_tests, 64), dtype=np.uint8)
    keys = np.random.randint(0, 2, size=(num_tests, 80), dtype=np.uint8)
    
    start_time = time.time()
    
    for i in range(num_tests):
        _ = encrypt(plaintexts[i:i+1], keys[i:i+1], rounds)
    
    end_time = time.time()
    total_time = end_time - start_time
    avg_time_per_test = total_time / num_tests
    
    print(f"Benchmark Results:")
    print(f"  Total time for {num_tests} encryptions: {total_time:.6f} seconds")
    print(f"  Average time per encryption: {avg_time_per_test:.6f} seconds")

# Run the benchmark
benchmark_present80(num_tests=1000, rounds=32)


Benchmark Results:
  Total time for 1000 encryptions: 0.866482 seconds
  Average time per encryption: 0.000866 seconds


In [ ]:
# Cipher KATAN
import numpy as np
from os import urandom

plain_bits = 32
key_bits = 80
word_size = 16

if plain_bits == 32:
    LEN_L1 = 13
    LEN_L2 = 19
    X = (None, 12, 7, 8, 5, 3)
    Y = (None, 18, 7, 12, 10, 8, 3)
elif plain_bits == 48:
    LEN_L1 = 19
    LEN_L2 = 29
    X = (None, 18, 12, 15, 7, 6)
    Y = (None, 28, 19, 21, 13, 15, 6)
else:
    LEN_L1 = 25
    LEN_L2 = 39
    X = (None, 24, 15, 20, 11, 9)
    Y = (None, 38, 25, 33, 21, 14, 9)

IR = (
    1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
    0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
    1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
    0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
    0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
    1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
    0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
    1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
    0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
    1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
    1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
    0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1,
    1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
    0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
    0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
    1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0
    )

MASK_VAL = 2 ** plain_bits - 1;


def encrypt(plaintext = [], K = [], nr = 0):
  P = np.flip(plaintext, axis = 1)
  ks = np.zeros(shape = (len(P), np.max([2*nr, 80])), dtype = np.uint8)
  ks[:, :80]  = np.flip(K, axis=1)
  for i in range(80, nr*2):
      ks[:, i] = ks[:, i-80] ^ ks[:, i-61] ^ks[:, i-50] ^ks[:, i-13]  
  for i in range(nr):
      fa = P[:, LEN_L2 + X[1]]^P[:, LEN_L2+ X[2]] ^ ks[:, 2*i] ^ (P[:,LEN_L2+ X[3]] & P[:, LEN_L2+X[4]]) ^ (P[:,LEN_L2+ X[5]] & IR[i])
      fb = P[:, Y[1]]^P[:, Y[2]] ^ (P[:, Y[3]] & P[:, Y[4]]) ^ (P[:, Y[5]] & P[:, Y[6]]) ^ ks[:, 2*i+1]
      P = np.roll(P, 1, axis=1)
      P[:, 0] = fa
      P[:, LEN_L2] = fb

  return np.flip(P, axis=1)

def convert_to_binary(arr):
  X = np.zeros((len(arr) * plain_bits,len(arr[0])),dtype=np.uint8);
  for i in range(len(arr) * plain_bits):
    index = i // plain_bits;
    offset = plain_bits - (i % plain_bits) - 1;
    X[i] = (arr[index] >> offset) & 1;
  X = X.transpose();
  return(X);

def convert_from_binary(arr, _dtype=np.uint64):
  num_words = arr.shape[1]//plain_bits
  X = np.zeros((len(arr), num_words),dtype=_dtype);
  for i in range(num_words):
    for j in range(plain_bits):
        pos = plain_bits*i+j
        X[:, i] += 2**(plain_bits-1-j)*arr[:, pos]
  return(X);



def check_testvectors():
    # From https://github.com/differential-neural/An-Assessment-of-Differential-Neural-Distinguishers/blob/main/cipher/katan.py
    p = np.zeros((2, plain_bits), dtype = np.uint8)
    p[1]^=1
    k = np.zeros((2, key_bits), dtype = np.uint8)
    k[0]^=1
    C = convert_from_binary(encrypt(p,k,254)).flatten()
    assert hex(C[0]) == '0x7e1ff945'  
    assert hex(C[1]) == '0x432e61da'  


check_testvectors()


In [3]:
# from ciphers.gpu import present80_gpu
import ciphers.cpu as ntan
import numpy as np
import cupy as cpy
import tensorflow as tf
from models import dbitnet
from models import train_nets as tn
import logging
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

ABORT_TRAINING_BELOW_ACC = 0.505   # if the validation accuracy reaches or falls below this limit, abort further training.
EPOCHS = 10                        # train for 10 epochs
NUM_SAMPLES = 10**7                 # create 10 million training samples
NUM_VAL_SAMPLES = 10**6             # create 1 million validation samples
BATCHSIZE = 5000   

def cupy_to_numpy_chunked(cupy_array, chunk_size=1_000_000):

    """
    Transfer a CuPy array to NumPy in chunks to avoid out-of-memory errors.
    
    Args:
        cupy_array: CuPy array to transfer.
        chunk_size: Number of elements per chunk (default: 1,000,000).
    
    Returns:
        NumPy array containing the transferred data.
    """
    total_size = cupy_array.size
    if total_size == 0:
        return np.array([])
    
    # Reshape to 1D for consistent slicing
    cupy_array_flat = cupy_array.ravel()
    result = []
    
    for start in range(0, total_size, chunk_size):
        end = min(start + chunk_size, total_size)
        # Transfer chunk to NumPy
        chunk = cupy_array_flat[start:end].get()
        result.append(chunk)
        # Synchronize to ensure GPU memory is freed
        cpy.cuda.Stream.null.synchronize()
    
    # Concatenate chunks and reshape to original shape
    numpy_array = np.concatenate(result)
    return numpy_array.reshape(cupy_array.shape)

def train_with_gpu():
    from ciphers.cpu import present80
    # from ciphers.cpu import present80_gpu

    encryption_function = present80.encrypt
    plain_bits = present80.plain_bits
    key_bits = present80.key_bits
    num_samples = 10_000_000
    num_samples_val = 1_00_000

    nr = 5

    def integer_to_binary_array(int_val, num_bits):
        return cpy.array([int(i) for i in bin(int_val)[2:].zfill(num_bits)], dtype = cp.uint8).reshape(1, num_bits)

    delta = cpy.integer_to_binary_array(0x90000000000, plain_bits)
    delta_key = 0
    delta_plain = delta[:plain_bits]

    print("hello world")

    data_generator = lambda num_samples, nr: ntan.make_train_data(
        encryption_function, plain_bits, key_bits, num_samples, nr, delta_plain, delta_key
    )

    C_cpu, Y_cpu = ntan.benchmark_make_train_data_gpu(encryption_function, num_samples, nr, plain_bits, key_bits)
    C_cpu_val, Y_cpu_val = ntan.benchmark_make_train_data_gpu(encryption_function, num_samples_val, nr, plain_bits, key_bits)

# get model architecture
    input_size = plain_bits
    model = dbitnet.make_model(2 * input_size)
    optimizer = tf.keras.optimizers.Adam(amsgrad=True)
    model.compile(optimizer=optimizer, loss='mse', metrics=['acc'])
    model.summary()

    # Transfer data in chunks
    chunk_size = 1_000_000  # 1M elements per chunk
    X = cupy_to_numpy_chunked(C_gpu, chunk_size)
    del C_gpu  # Free GPU memory
    cpy.cuda.Stream.null.synchronize()
    Y = cupy_to_numpy_chunked(Y_gpu, chunk_size)
    del Y_gpu
    cpy.cuda.Stream.null.synchronize()
    X_val = cupy_to_numpy_chunked(C_gpu_val, chunk_size)
    del C_gpu_val
    cpy.cuda.Stream.null.synchronize()
    Y_val = cupy_to_numpy_chunked(Y_gpu_val, chunk_size)
    del Y_gpu_val
    cpy.cuda.Stream.null.synchronize()
    print(X.shape)
    # Clear TensorFlow session
    tf.keras.backend.clear_session()
    model_name= 'dbinet_gpu_dataset'
    val_acc = tn.train_one_round(
        model,
        X, Y, X_val, Y_val,
        round_number = nr,
        epochs=EPOCHS,
        log_prefix="./",
        model_name=model_name,
        LR_scheduler=1e-5,
    )
    print(f'{model_name}, round {nr}. Best validation accuracy: {val_acc}', flush=True)

def train_with_cpu():
    from ciphers.cpu import present80

    encryption_function = present80.encrypt
    plain_bits = present80.plain_bits
    key_bits = present80.key_bits
    num_samples = 10_000_000
    num_samples_val = 1_000_000

    nr=8

    delta = ntan.integer_to_binary_array(0x90000000000, plain_bits)
    delta_key = 0
    delta_plain = delta[:plain_bits]

    data_generator = lambda num_samples, nr: ntan.make_train_data(
        encryption_function, plain_bits, key_bits, num_samples, nr, delta_plain, delta_key
    )

    C_gpu, Y_gpu = ntan.benchmark_make_train_data_cpu(encryption_function, num_samples, nr, plain_bits, key_bits)
    C_gpu_val, Y_gpu_val = ntan.benchmark_make_train_data_cpu(encryption_function, num_samples_val, nr, plain_bits, key_bits)


# get model architecture
    input_size = plain_bits
    model = dbitnet.make_model(2 * input_size)
    optimizer = tf.keras.optimizers.Adam(amsgrad=True)
    model.compile(optimizer=optimizer, loss='mse', metrics=['acc'])
    model.summary()

    model_name= 'dbinet_gpu_dataset'
    val_acc = tn.train_one_round(
        model,
        C_gpu, Y_gpu, C_gpu_val, Y_gpu_val,
        round_number = nr,
        epochs=EPOCHS,
        log_prefix="./",
        model_name=model_name,
        LR_scheduler=1e-5,
    )
    print(f'{model_name}, round {nr}. Best validation accuracy: {val_acc}', flush=True)

if __name__ == "__main__":
    train_with_cpu()

ModuleNotFoundError: No module named 'ciphers'

In [2]:
!pip list

Package                           Version
--------------------------------- --------------
absl-py                           2.1.0
anyio                             4.4.0
argon2-cffi                       23.1.0
argon2-cffi-bindings              21.2.0
arrow                             1.3.0
asgiref                           3.8.1
aspose-words                      24.9.0
asttokens                         2.4.1
astunparse                        1.6.3
async-lru                         2.0.4
attrs                             24.2.0
babel                             2.16.0
beautifulsoup4                    4.12.3
bleach                            6.1.0
caer                              2.0.8
certifi                           2024.8.30
cffi                              1.17.1
charset-normalizer                3.3.2
click                             8.1.8
collab-utils                      0.0.1
colorama                          0.4.6
comm                              0.2.2
contourpy         